# Exercise 10

Remember that a discrete dynamical system involves an $N\times N$ "transition" matrix $\mathbf{A}$ and an $N\times 1$ state vector $\mathbf{u}_k$ at time $k$:
$$
\begin{equation}
\mathbf{u}_k = \mathbf{A}\mathbf{u}_{k-1} = \mathbf{A}^k\mathbf{u}_{0}
\end{equation}
$$
given the $N^A \le N$ eigenvalues $\lambda_i$ and corresponding (orthonormal) eigenvectors $\mathbf{x}_i$ of the matrix $\mathbf{A}$, the initial state vector can be projected onto the eigenspace of $\mathbf{A}$:
$$
\begin{equation}
\mathbf{u}^A_0 = proj(\mathbf{u}_0) = \sum_{i=1}^{N^A} a_i\mathbf{x}_{i}
\end{equation}
$$
where $a_i$ are a set of $N^A$ constants and the eigenvectors of $\mathbf{A}$ do not necessarily need to span $\mathbb{R}^N$. Substituting the decomposed $\mathbf{u}_0$ into the original formulation:

\begin{equation}
\mathbf{u}_k =  \mathbf{A}^k \mathbf{u}_{0} = \sum_{i=1}^{N^A} a_i\lambda^k \mathbf{x}_{i}
\end{equation}

Create a Python module `dds.py` that uses the `numpy` and `matplotlib` modules to evaluate and visualize discrete dynamical systems. The module should contain the following functions:

* `eigendecompose(A, u0)` -- returns a tuple `(evals, evecs, consts)` where `evals` is the `list` of eigenvalues and `evecs` is the `list` ([$\mathbf{x}_1, \ \mathbf{x}_2, \ ..., \mathbf{x}_{N^A}$]) of eigenvectors of the `numpy` matrix `A` and `consts` is a `list` of constants $a_i$ from above for `u0`. Note that the eigenvectors should form an orthonormal set (if possible!).
* `evaluate_decomposed(evals, evecs, consts, k)` -- *optimally* evaluates and returns an array $u_k$ given a list of eigenvalues, a list of eigenvectors, a list of constants, and $k$.
* `evaluate(A, u0, k)` -- *optimally* evaluates and returns an array $u_k$ given an $N\times N$ transition matrix and $N\times 1$ initial state vector.
* `evolution(A, u0, k)` -- returns an `numpy.ndarray` `U` with shape $(N, k+1)$ which contains each $u_k$ for $k=0\rightarrow k$ given an $N\times N$ transition matrix and $N\times 1$ initial state vector.
* `plot(U, labels=None)` -- returns a `figure` object representing a single plot including each of the components of the state vectors contained in `U` with respect to step number ($k$). If `labels` is a `list` of $N$ strings, the figure should contain a legend with labels from the list.

All functions should check for valid input arguments and include appropriate doc-strings. If the input is invalid, the function should return `None`.

In [ ]:
"""Starting with the first function def eigendecompose(A, u0)
This function needs to return the eigenvalues and eigenvectors
of a given matrix, A, and the constants, a_i, for a given initial
state vector, u0. Here's nhow that would look
"""
import numpy as np

def eigendecompose(A, u0):
    """
    The goal of this function is to decompose the initial state 
    vector, u0, into its components in the eigenspace of A.

    Parameters:
    A (np.ndarray): Transition matrix
    u0 (np.ndarray): Initial state vector

    Returns:
    tuple: specifically a tuple containing the eigenvalues, 
    eigen vectors and constants.
    """
    # Step 1, Check if input is vaild
    if not isinstance(A, np.ndarray) or not isinstance(u0, np.ndarray):
        return None
    #we should indeed check if the inputs are valid. Specifically, 
    #we should check if A is a square matrix (since only square matrices
    #have eigenvalues and eigenvectors), and if u0 is a column vector 
    #with a number of rows that matches the number of columns in A.
    elif A.shape[0] != A.shape[1] or A.shape[1] != u0.shape[0]:
        return None
    
    #Step 2, Calculate eigenvalues and eigenvectors
    evals, evecs = np.linalg.eig(A)
    
    #Normalize the evecs
    evecs = evecs / np.linalg.norm(evecs, axis=0)
    #We must transpose to get each eigenvector as a row
    evecs = np.transpose(evecs) 

    #Step 3, Now calculate the constants
    consts = np.linalg.solve(evecs, u0)

    """
    Now to calculate the constants, or find our constants in
    relation to solving the system: evecs * const = u0, in order
    to find the constants that decompose u0 into the basis of
    eigenvectors. We use np.linalg.solve for this, which in turn
    solves the linear system.

    Another change is transposing the eigenvectors before 
    converting it to a list of lists.  This way, each eigenvector
    will be a separate sublist, which is likely the format 
    expected for the evaluation.
    """

   
    """
    In this version, we convert the eigenvalues and constants
    to lists using the .tolist() method.
    """
    return evals.tolist(), [evec for evec in evecs], consts.tolist()

"""
This first function checks if the inputs are vaild numpy 
arrays.  If not, it returns None.  It then calculates the
eigenvalues and eigen vectors using the numpy.linalg.eig
function.  The constants are calculated by projecting the 
intial state vector onto the eigenspace of A, which is done
by multiplying the transpose of the eigenvectors matrix with
the initial state vector.
"""

"""
For our next function in the line up: 
def evaluate_decomposed(evals, evecs, consts, k).  This one 
will take in the eigenvalues, eigenvectors, and constants we 
calculated, along with a time step, k, and return the state
vector at time, k.  Here's a basic structure of how we could
write this function:
"""
def evaluate_decomposed(evals, evecs, consts, k):
    """
    Objective of function:
    Evaluates the state vector at time k given the; eigenvalues,
    eigenvectors, and constants

    Parameters:
    evals (class: 'list'): Eigenvalues of the transition matrix
    evecs (class: 'list'): Eigenvectors of the transition matrix
    consts (class: 'list'): Constants for initial state vector 
    decompostion
    k (class: 'int'): Time step

    Returns:
    np.ndarray: State vector at time, k.
    """
    #Step 1, check if input is vaild
    if not all(isinstance(i, (list, np.ndarray)) for i in [evals, evecs, consts]) or not isinstance(k, int):
        return None

    #Convert to numpy arrays for efficient calculation
    evals, evecs, consts = np.array(evals), np.array(evecs), np.array(consts)

    #Step 2, calculate state vector at time, k.
    uk = np.sum(consts * (evals**k)[:, None]*evecs, axis=0) 
    #Instead of directly multiplying the arrays, we can use the numpy function 
    #np.multiply which performs element-wise multiplication:
    return uk

"""
This second function calculates the state vector as time, k, by
summing up the products of each; constant, its corresponding
eigenvalue raised to the power, k, and its corresponding 
eigenvector.
"""

"""
Now for the third function: def evaluate(a, u0, k).  This next
function will now evaluate the state vector at time, k, given a
transition matrix, A, and an intial state vector, u0. Here's a 
basic structure of how this fuction could look:
"""
def evaluate(A, u0, k):
    """
    Function objective:
    evaluates the state vector at time, k, given a transition
    matrix and an initial state vector.

    Parameters:
    A (np.ndarray): Transition matrix
    u0 (np.ndarray): Initial state vector
    k (int): Time step

    Returns:
    np.ndarray: State vector at time k
    """

    #Step 1, check if input is vaild
    if not isinstance(A, np.ndarray) or not isinstance(u0, np.ndarray) or not isinstance(k, int):
        return None
    #Check if A is a square matrix and u0 has the same number of rows as A has columns
    elif A.shape[0] != A.shape[1] or A.shape[0] != A.shape[1] != u0.shape[0]:
        return None

    #Step 2, Calculate state vector at time, k
    uk = np.array(np.linalg.matrix_power(A, k).dot(u0))
    #The np.dot() function in the evaluate function returns a numpy 
    #array by default. However, it's always good to ensure that 
    #the output is indeed an array.

    return uk

"""
This function calculates the state vector at time, k, by
raising the transition matrix, A, and then multiplying the 
result with the initial state vector, u0.
"""

"""
Now to the next fctn: def evolution(A,u0,k) function.  This fctn
should return an array with each state vector from time, k = 0
to the upper limit, k = n (0 to k), given a transition matrix,
A, and an initial state vector u0.  Here's a basic structure on
how this fctn would work:
"""
def evolution(A, u0, k):
    """
    Fctn objective:
    Returns an array with each state vector from time 0 to time k.

    Parameters:
    A (np.ndarray): Transistion matrix
    u0 (np.ndarray): intial state vector
    k (int): Time step

    Returns:
    np.ndarray: Array containing each state vector from time 0 to 
    time k
    """

    #Check if input is vaild
    if not isinstance(A, np.ndarray) or not isinstance(u0, np.ndarray) or not isinstance(k, int):
        return None

    #Initialize array with initial state vector
    U = np.empty((len(u0),k+1))
    U[:, 0] = u0.flatten()

    #Calculate each state vector and add it to the array
    for i in range(1, k+1):
        U[:,i] = np.dot(A, U[:, i-1])

    return U
"""
This function initializes an array U with the initial state 
vector, u0.  It then calculates each subsequent state vector 
by multiplying the transition matrix A with the previous state
vector and adds it to the array.
"""

"""
And now for the final function: def plot(U, labels=None).
the plot(U, labels=None) fctn will create a plot of the 
evolution of the state of vectors.  Here's a basic structure
of what that would look like.
"""

import matplotlib.pyplot as plt

def plot(U, labels=None):
    """
    Plots the components of the state vectors in U with respect to time, k.

    Parameters:
    U (np.ndarray): Array containing each state vector from time 0 to time k.
    labels (list): List of labels for the components of the state vectors.

    Returns:
    figure: a matplotlib figure object
    """
    # Create figure and axis
    fig, ax = plt.subplots()

    # Plot each component of the state vectors
    for i in range(U.shape[0]):
        if labels:
            ax.plot(range(U.shape[1]))
        else:
            ax.plot(range(U.shape[1]), U[i, :])
    # Display the labels if provided
    if labels:
        ax.legend()

    return fig
